## cx网站练习数据爬取

In [ ]:
import requests
import json
import time 
import pandas as pd
from bs4 import BeautifulSoup

from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from requests.exceptions import RequestException
import re
import pandas as pd

def get_one_page(url):
    headers = {
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3135.4 Safari/537.36"
    }
    try:
        response=requests.get(url,headers=headers)
        if(response.status_code==200):
            response.encoding='utf-8'
            return response.text
        return None
    except RequestException:
        print("error")
        return None 

def get_next_url(url):
    html=get_one_page(url)
    soup=BeautifulSoup(html,'lxml')
    try:
        next_page=soup.select('a[href]')
        next_url=next_page[51]['href']
        next_url='http://cxmooc.hustoj.com/'+next_url
        return next_url
    except:
        return None

def parse_one_page_and_write_as_csv(file_path,counts):
    run_numbers=[]
    users=[]
    questions=[]
    results=[]
    memorys=[]
    time_consuming=[]
    language=[]
    code_length=[]
    submit_time=[]
    question_machine=[]
    url='http://cxmooc.hustoj.com/status.php?'
    for i in range(counts):
        html=get_one_page(url)
        url=get_next_url(url)
        if(url==None):
            break
        soup=BeautifulSoup(html,'lxml')
        comment_lists=soup.find('tbody')
        trs=comment_lists.select('tr')

        for tr in trs:
            td=tr.select('td')
            run_numbers.append(td[0].get_text())
            users.append(td[1].get_text())
            questions.append(td[2].get_text())
            results.append(td[3].get_text())
            memorys.append(td[4].get_text())
            time_consuming.append(td[5].get_text())
            language.append(td[6].get_text())
            code_length.append(td[7].get_text())
            submit_time.append(td[8].get_text())
            question_machine.append(td[9].get_text())

    data=[run_numbers,users,questions,results,memorys,time_consuming,language,code_length,submit_time,question_machine]
    result=pd.DataFrame(data)
    result=result.T
    result.columns=['运行编号','用户','问题','结果','内存','耗时','语言','代码长度','提交时间','判题机']
    result.to_excel(file_path)
    

def main(file_path,counts):
    parse_one_page_and_write_as_csv(file_path,counts)
        
if __name__=='__main__':
    file_path='F:\hua\mo\陈老师.xlsx'
    main(file_path,3000)

## 进行数据分析

In [1]:
import pandas as pd
data=pd.read_excel('F:\hua\mo\陈老师.xlsx')
data=data[(data['语言']=='C') | (data['语言']=='C++')]#保留c跟c++类型的数据
data=data.loc[:,['运行编号','用户','问题','结果','代码长度','提交时间']]
data=data[data['运行编号']>=91688]
data['代码长度']=data['代码长度'].apply(lambda x:x[:(len(x)-2)])#去除代码长度的单位B

In [2]:
def student(data):
    users=data['用户'].unique()
    users_bili=[]
    for user in users:
        data_user=data[data['用户']==user]
        data_user=data_user[data_user['结果']=='正确']#保留结果为正确的题目来分析
        data_user=data_user.drop_duplicates(['问题'])#将问题去重
        daima_range=data_user['代码长度'].unique()
        result= data_user.groupby([data_user['代码长度']])['问题'].count()
        if(result[result>1].sum()==0 & result.sum()==0):
            bili==0
        else:
            bili=result[result>1].sum()/result.sum()
        users_bili.append(bili)
    data_reault=pd.DataFrame(users_bili,users)
    data_reault.to_csv('F:\hua\mo\data_reault.csv',encoding='utf-8')
    data_reault=pd.read_csv('F:\hua\mo\data_reault.csv')
    data_reault.columns=['用户','比例']
    data_reault=data_reault.sort_values(['比例'],ascending=False)
    data_reault.to_excel('F:\hua\mo\data_reault.xlsx')

student(data)

In [3]:
def time_jiange(data):
    data['提交时间']=data['提交时间'].apply(lambda x : x[:10])
    users=data['用户'].unique()
    users_id=[]
    usesr_count=[]
    zong=[]
    zuida=[]
    for user in users: 
        users_id.append(user)
        data_user=data[data['用户']==user]
        data_user['提交时间'] = pd.to_datetime(data_user['提交时间'])
        c=data_user['提交时间']
        jiantian=pd.Series('2017-11-27 00:00:00')
        c=jiantian.append(c)
        zong.append(len(c))
        c=c.drop_duplicates()
        jiange=[]
        for i in range(len(c)-1):
            a=pd.to_datetime(c.iloc[i])-pd.to_datetime(c.iloc[i+1])
            b=int(a.total_seconds()/(60*60*24))
            jiange.append(b)
        count=0
        da=max(jiange)
        zuida.append(da)
        for i in jiange:
            if(i>=3):
                count+=1
        usesr_count.append(count)
    result=pd.DataFrame([users_id,usesr_count,zong,zuida])
    result=result.T
    result.columns=['用户','间隔3天及其3天以上的次数','总提交次数','最大时间间隔']
    result=result.sort_values(['间隔3天及其3天以上的次数'],ascending=False)
    result.to_excel('F:\\hua\\mo\\result.xlsx')
time_jiange(data)

C:\Users\pujing\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
